In [ ]:
import time
print(time.localtime()) #returns the local time at the start of execution. Can be used to calculate the execution time manually

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
import scipy.signal as sig
import control as con
from statsmodels.tsa.arima_model import ARIMA
import warnings
%pylab inline
warnings.filterwarnings('ignore') #ignores all the warnings that may arise during runtime

###### Reading all the data

In [ ]:
#data1
df_soll_1 = pd.read_csv('step_log/2017-11-03_ext_topic_logger/2017-11-03_14-25-18_control_output.log',
                 header=0,
                 names=['time', 'x_soll'])
df_soll_1 = df_soll_1.set_index('time')
df_soll_1 = df_soll_1[~df_soll_1.index.duplicated(keep='first')] #gets rid of any duplicate values if present
df_ist_1 = pd.read_csv('step_log/2017-11-03_ext_topic_logger/2017-11-03_14-25-19_task_vel.log',
                 header=0,
                 names=['time', 'x_ist'])
df_ist_1 = df_ist_1.set_index('time')
df_ist_1 = df_ist_1[~df_ist_1.index.duplicated(keep='first')]
df_ist_soll_1 = pd.concat([df_soll_1.x_soll, df_ist_1.x_ist], axis=1).fillna(method='pad')
df_ist_soll_1 = df_ist_soll_1.fillna(0)
#df_ist_soll_1.plot(style='-', drawstyle="steps")
#data2
df_soll_2 = pd.read_csv('step_log/2017-11-03_ext_topic_logger/2017-11-03_14-26-43_control_output.log',
                 header=0,
                 names=['time', 'x_soll'])
df_soll_2 = df_soll_2.set_index('time')
df_ist_2 = pd.read_csv('step_log/2017-11-03_ext_topic_logger/2017-11-03_14-26-43_task_vel.log',
                 header=0,
                 names=['time', 'x_ist'])
df_ist_2 = df_ist_2.set_index('time')
df_ist_2 = df_ist_2[~df_ist_2.index.duplicated(keep='first')]
df_ist_soll_2 = pd.concat([df_soll_2.x_soll, df_ist_2.x_ist], axis=1).fillna(method='pad')
df_ist_soll_2 = df_ist_soll_2.fillna(0)
#df_ist_soll_2.plot(style='-', drawstyle="steps")
#data3
df_soll_3 = pd.read_csv('step_log/2017-11-03_ext_topic_logger/2017-11-03_14-27-47_control_output.log',
                 header=0,
                 names=['time', 'x_soll'])
df_soll_3 = df_soll_3.set_index('time')
df_soll_3 = df_soll_3[~df_soll_3.index.duplicated(keep='first')] 
df_ist_3 = pd.read_csv('step_log/2017-11-03_ext_topic_logger/2017-11-03_14-27-47_task_vel.log',
                 header=0,
                 names=['time', 'x_ist'])
df_ist_3 = df_ist_3.set_index('time')
df_ist_3 = df_ist_3[~df_ist_3.index.duplicated(keep='first')]
df_ist_soll_3 = pd.concat([df_soll_3.x_soll, df_ist_3.x_ist], axis=1).fillna(method='pad')
df_ist_soll_3 = df_ist_soll_3.fillna(0)
#df_ist_soll_3.plot(style='-', drawstyle="steps")
#data4
df_soll_4 = pd.read_csv('step_log/2017-11-03_ext_topic_logger/2017-11-03_14-30-43_control_output.log',
                 header=0,
                 names=['time', 'x_soll'])
df_soll_4 = df_soll_4.set_index('time')
df_ist_4 = pd.read_csv('step_log/2017-11-03_ext_topic_logger/2017-11-03_14-30-43_task_vel.log',
                 header=0,
                 names=['time', 'x_ist'])
df_ist_4 = df_ist_4.set_index('time')
df_ist_4 = df_ist_4[~df_ist_4.index.duplicated(keep='first')]
df_ist_soll_4 = pd.concat([df_soll_4.x_soll, df_ist_4.x_ist], axis=1).fillna(method='pad')
df_ist_soll_4 = df_ist_soll_4.fillna(0)
#df_ist_soll_4.plot(style='-', drawstyle="steps")

###### Removing all zeros & the negative trend and reformatting the data in accordance with a unit step response

In [ ]:
#function that strips zeros and multiplies the dataframe to 1
def strip_multiply(dataframe):
    no_zero_df                     = dataframe[dataframe.x_soll > 0] #selects the dataframe that has no zeros in x_soll
    time_of_step                   = no_zero_df.index[0] #returns the time at which the step occurs
    last_zero_df                   = dataframe[(dataframe.x_soll == 0) & (dataframe.index < time_of_step)].tail(1) #selects the df containing last zero value before the change in step 
    
    
    response_start_time            = pd.concat([last_zero_df, no_zero_df]).index[0] #returns the actual starting time of the dataframe
    data_with_initial_zero         = pd.concat([last_zero_df, no_zero_df]) #returns a dataframe that starts from a 0 value so that we get an actual'step'
    time_series_starting_from_zero = data_with_initial_zero.index-response_start_time #returns a new time series data whose time starts from zero
    data_for_modeling              = data_with_initial_zero.set_index(time_series_starting_from_zero) #changing the index of the dataframe with the new time series

    x_soll_steady_state            = no_zero_df.x_soll.head(1) #returns the steady state value along with the time index
    multiplication_factor          = 1 / (pd.Series(x_soll_steady_state).values[0]) #calculates the factor to be multiplied with each of the data
    input_array                    = (multiplication_factor * data_for_modeling.x_soll).tolist() #returns the 1D array  after multiplying the input with the factor in order to equalise it with 1 
    output_array                   = (multiplication_factor * data_for_modeling.x_ist).tolist() #returns the 1D array  after multiplying the output with the factor in order to equalise it with 1
    time_array                     = data_for_modeling.index.tolist() #extracting the time series index into a 1D array
    return input_array, output_array, time_array



#data1
df_ist_soll_1 = df_ist_soll_1[(df_ist_soll_1.x_ist > 0)]
xin_1, yout_1, t_1 = strip_multiply(df_ist_soll_1)
#plt.plot(t_1,yout_1)
#plt.plot(t_1,xin_1)


#data2
df_ist_soll_2 = df_ist_soll_2[(df_ist_soll_2.x_ist > 0)]
xin_2, yout_2, t_2 = strip_multiply(df_ist_soll_2)  
#plt.plot(t_2,yout_2)
#plt.plot(t_2,xin_2)


#data3-data with a special case trend
df_ist_soll_3 = df_ist_soll_3[(df_ist_soll_3.x_ist > 0)]
#the following code identifies the decreasing trend in x_ist values and removes those values from the dataframe
df_ist_soll_3['trend'] = np.sign(df_ist_soll_3['x_ist'].rolling(window = 5).mean().diff().fillna(0)).map({0:'FLAT',1:'UP',-1:'DOWN'})
reversed_array = list(df_ist_soll_3.trend.values)[::-1]
counter = 0
for i in range(0,len(reversed_array)):
    if reversed_array[i] == 'DOWN':
        counter = counter+1
    else:
        break
length = len(df_ist_soll_3)
df_ist_soll_3 = df_ist_soll_3.head(length - counter)
xin_3, yout_3, t_3 = strip_multiply(df_ist_soll_3)  
#plt.plot(t_3,yout_3)
#plt.plot(t_3,xin_3)


#data4
df_ist_soll_4 = df_ist_soll_4[(df_ist_soll_4.x_ist > 0)]
xin_4, yout_4, t_4 = strip_multiply(df_ist_soll_4)  
#plt.plot(t_4,yout_4)
#plt.plot(t_4,xin_4)


yout = [yout_1, yout_2, yout_3, yout_4]
t = [t_1, t_2, t_3, t_4]

###### Section for getting the fitted model and aic values 

In [ ]:
%load_ext rpy2.ipython

In [ ]:
%%R 
require(ggplot2)
library(stats)
library(sysid)
library(forecast)

In [ ]:
def order_ar(ar_order,yout):
    %R -i ar_order,yout
    %R yout = unlist(yout)                                           #added newly in R version 3.4.2
    %R sysar = ar(yout,method="ols",order.max=ar_order)
    %R far = fitted.values(sysar)
    %R -o far
    far_new=np.nan_to_num(far)
    %R sysar_new = sysar[2]
    %R -o sysar_new
    mse_ar = mean_squared_error(yout, far_new)
    np_ar = len(sysar_new[0])
    %R -i mse_ar
    %R -i np_ar 
    %R N = length(yout) 
    %R aic_ar = N*log(mse_ar) + 2*np_ar + N*dim(matrix(yout))[2]*(log(2*pi)+1)
    %R -o aic_ar,mse_ar
    return aic_ar, mse_ar, far_new

###### MSE and AIC Dataframe for orders 1 to 10

In [ ]:
#The function returns a dataframe that contains the aic,mse and order values from 1 to 10 along with the fitted values for each order 
def order_aic_mse_fit(yout):
    aic                  = []
    order                = []
    mse                  = []
    fit_array            = []
    aic_mse_fit_df       = [] 
    aic_mse_fit_df.append([])#2D array in which each element stores two floats(aic and mse) and an array(fit_array)
   
    for i in range(1,11):
        order.append(i)
        aic_mse_fit_df.append(order_ar(i,yout))
        aic.append(list(aic_mse_fit_df[i][0]))
        mse.append(aic_mse_fit_df[i][1])
        fit_array.append(aic_mse_fit_df[i][2])
    
    df = pd.DataFrame(np.column_stack([order, aic, mse]),columns=['order', 'aic', 'mse']) #all variables are passed into the dataframe as type float by default  
    return df, fit_array
    

df_1, fit_array_1 = order_aic_mse_fit(yout_1)
df_2, fit_array_2 = order_aic_mse_fit(yout_2)
df_3, fit_array_3 = order_aic_mse_fit(yout_3)
df_4, fit_array_4 = order_aic_mse_fit(yout_4)

###### Selecting the best order and the corresponding aic, mse and fitted values for each data based on low mse value

In [ ]:
order_1, mse_ar_1, aic_ar_1 = list(df_1[df_1.mse == df_1.mse.min()].values[0]) #list function lists the df row having the least mse into a 1D array 
fitted_1 = fit_array_1[int(order_1)-1]#int function converts the order back to the integer value. Order-1 gives the corresponding index value 

order_2, mse_ar_2, aic_ar_2  = list(df_2[df_2.mse == df_2.mse.min()].values[0])
fitted_2 = fit_array_2[int(order_2)-1]

order_3, mse_ar_3, aic_ar_3 = list(df_3[df_3.mse == df_3.mse.min()].values[0])
fitted_3 = fit_array_3[int(order_3)-1]

order_4, mse_ar_4, aic_ar_4  = list(df_4[df_4.mse == df_4.mse.min()].values[0])
fitted_4 = fit_array_4[int(order_4)-1]

#plt.plot(t_4,yout_4,label='original')
#plt.plot(t_4,fitted_4,label='model')
#plt.legend()

###### Smoothing all the model outputs

In [ ]:
def smooth1(model):
    y_ar=model.ravel()
    c = 0
    for i in range(0,len(y_ar)):
        if y_ar[i]<=0.01:
            c = c + 1
        else:
            break

    y_ar_new = y_ar[c:]
    if len(y_ar_new)%2!=0:
        d_ar = sig.savgol_filter(y_ar_new,round(len(y_ar_new)/2)+1,1)
    elif len(y_ar_new)%4==0:
        d_ar = sig.savgol_filter(y_ar_new,round(len(y_ar_new)/2)+1,1)
    else:
        d_ar = sig.savgol_filter(y_ar_new,round(len(y_ar_new)/2),1)
    smoothed = np.append(y_ar[0:c],d_ar)
    return smoothed      
def smooth2(model):
    y_ar=model.ravel()
    c = 0
    for i in range(0,len(y_ar)):
        if y_ar[i]<=0.01:
            c = c + 1
        else:
            break

    y_ar_new = y_ar[c:]
    if len(y_ar_new)%2!=0:
        d_ar = sig.savgol_filter(y_ar_new,round(len(y_ar_new)/2)+1,2)
    elif len(y_ar_new)%4==0:
        d_ar = sig.savgol_filter(y_ar_new,round(len(y_ar_new)/2)+1,2)
    else:
        d_ar = sig.savgol_filter(y_ar_new,round(len(y_ar_new)/2),2)
    smoothed = np.append(y_ar[0:c],d_ar)
    return smoothed      
smooth_11 = smooth1(fitted_1)
smooth_21 = smooth1(fitted_2)
smooth_31 = smooth1(fitted_3)
smooth_41 = smooth1(fitted_4)
smooth_12 = smooth2(fitted_1)
smooth_22 = smooth2(fitted_2)
smooth_32 = smooth2(fitted_3)
smooth_42 = smooth2(fitted_4)
#plt.plot(T_2,smooth_2,label='smoothed')
#plt.plot(T_2,fitted_2,label='model')
#plt.legend()

###### PT1 and PT2 Modeling on all data 

In [ ]:
def pt1(smooth,T):
    dar=pd.DataFrame(smooth,columns=['vall'])
    tim=pd.DataFrame(T,columns=['time'])
    dfo=pd.concat([dar,tim],axis=1)
    youtd_frame=pd.DataFrame(smooth,columns=['value'])
    yss=youtd_frame.tail(n=1)
    yss = pd.Series(yss)
    yss = pd.to_numeric(yss)
    yss=yss[0]
    ystd=yss*(1-np.exp(-1))
    t10 = dfo[dfo.vall>0.01].values[0][1]                 #was 0.0 in case of modeling with R
    a0 = 1
    a1 = dfo.time[dfo.index == abs(dfo.vall-ystd).sort_values().index[0]].values[0]
    b0 = yss
    tf = con.matlab.tf(b0, [a1, a0])
    n, d = con.pade(t10, 1)
    delay = con.matlab.tf(n,d)
    youto, to = con.matlab.step(tf*delay)
    return tf,delay,youto,to
def pt2(smooth,T):
    dar=pd.DataFrame(smooth,columns=['vall'])
    tim=pd.DataFrame(T,columns=['time'])
    dfo=pd.concat([dar,tim],axis=1)
    youtd_frame=pd.DataFrame(smooth,columns=['value'])
    yss=youtd_frame.tail(n=1)
    yss = pd.Series(yss)
    yss = pd.to_numeric(yss)
    yss=yss[0]
    td_frame=pd.DataFrame(T,columns=['time'])
    df = pd.concat([td_frame,youtd_frame],axis=1)
    yss1=yss*0.1
    yss3=yss*0.3
    yss6=yss*0.6
    yss9=yss*0.9
    #t1 = np.mean([df.time[df.index==abs(df.value-yss1).sort_values().value_counts().max()-1].values[0],df.time[df.index==abs(df.value-yss1).sort_values().value_counts().max()].values[0]])
    t1 = df.time[df.index == abs(df.value-yss1).sort_values().index[0]].values[0]
    t3 = df.time[df.index == abs(df.value-yss3).sort_values().index[0]].values[0]
    t6 = df.time[df.index == abs(df.value-yss6).sort_values().index[0]].values[0]
    t9 = df.time[df.index == abs(df.value-yss9).sort_values().index[0]].values[0]
    def fourpoint(z):
        f1_zeta = 0.451465 + 0.066696*z + 0.013639*z**2
        f3_zeta = 0.800879 + 0.194550*z + 0.101784*z**2
        f6_zeta = 1.202664 + 0.288331*z + 0.530572*z**2
        f9_zeta = 1.941112 - 1.237235*z + 3.182373*z**2
        return f1_zeta,f3_zeta,f6_zeta,f9_zeta
    beta = (t9 - t6)/(t3 - t1)
    zeta_est_beta = -0.460805 + 0.976315*beta - 0.254517*beta**2 + 0.028115*beta**3
    f1_zeta,f3_zeta,f6_zeta,f9_zeta = fourpoint(zeta_est_beta)
    peak1=youtd_frame.max()
    peak1 = peak1[0]
    overshoot = (peak1 - yss)/yss
    zeta_est_overshoot = numpy.sqrt(numpy.log(overshoot)**2 / (numpy.pi**2 + numpy.log(overshoot)**2))
    f1_zeta,f3_zeta,f6_zeta,f9_zeta = fourpoint(zeta_est_overshoot)
    def method2(z):
        T_est2 = (t9 - t1) / (f9_zeta - f1_zeta) 
        #theta_est2 = t1 - T_est2*f1_zeta              #based on research paper
        theta_est2 = T_est2*f1_zeta
        return T_est2,theta_est2
    T_est2,theta_est2 = method2(zeta_est_overshoot)     
    tf2 = con.matlab.tf(yss, [T_est2**2,2*zeta_est_overshoot*T_est2, 1])
    #n_2, d_2 = con.pade(theta_est2, 1)
    n_2, d_2 = con.pade(t1, 1)      # based on http://cse.lab.imtlucca.it/~bemporad/teaching/ac/pdf/AC2-08-System_Identification.pdf
    delay2 = con.matlab.tf(n_2,d_2)
    youto2,to2 = con.matlab.step(tf2*delay2)
    return tf2,delay2,youto2,to2

tf_11,delay_11,youto_11,to_11 = pt1(smooth_11,t_1)
tf_21,delay_21,youto_21,to_21 = pt1(smooth_21,t_2)
tf_31,delay_31,youto_31,to_31 = pt1(smooth_31,t_3)
tf_41,delay_41,youto_41,to_41 = pt1(smooth_41,t_4)
tf_12,delay_12,youto_12,to_12 = pt2(smooth_12,t_1)
tf_22,delay_22,youto_22,to_22 = pt2(smooth_22,t_2)
tf_32,delay_32,youto_32,to_32 = pt2(smooth_32,t_3)
tf_42,delay_42,youto_42,to_42 = pt2(smooth_42,t_4)
youto1 = [youto_11,youto_21,youto_31,youto_41]
to1 = [to_11,to_21,to_31,to_41]
youto2 = [youto_12,youto_22,youto_32,youto_42]
to2 = [to_12,to_22,to_32,to_42]
#plt.plot(to_2,youto_2)
#plt.plot(T_2,yout_2)

###### State space representation 

In [ ]:
def ss(TF):
    sys = con.matlab.tf2ss(TF)
    return sys
sys_11 = ss(tf_11*delay_11)
sys_21 = ss(tf_21*delay_21)
sys_31 = ss(tf_31*delay_31)
sys_41 = ss(tf_41*delay_41)
sys_12 = ss(tf_12*delay_12)
sys_22 = ss(tf_22*delay_22)
sys_32 = ss(tf_32*delay_32)
sys_42 = ss(tf_42*delay_42)
SYS1 = [sys_11,sys_21,sys_31,sys_41]
SYS2 = [sys_12,sys_22,sys_32,sys_42]

###### Mean square comparison of each data and its model

In [ ]:
def mse(df_ist_soll,yout,youtx,tx):
    Y = pd.concat([pd.DataFrame(list(df_ist_soll.index.values),columns=['time']),pd.DataFrame(yout,columns=['y_1'])],axis=1)
    X = pd.concat([pd.DataFrame(youtx,columns=['y_12']),pd.DataFrame(tx,columns=['time'])],axis=1)
    X1 = list(pd.Series(X.time))
    Y1 = list(pd.Series(Y.time))

    arr = []
    for i in range(0,len(X1)):
        arr.append([])
        for j in range(0,len(Y1)):
            if round(abs(X1[i]))==round(abs(Y1[j])) or (round(X1[i]) == round(Y1[j]+0.5)) or (round(X1[i]+0.5) == round(Y1[j])):
                arr[i].append(Y1[j])

    delayed = []
    for i in range(0,len(X1)):
        delayed.append(min(abs(X1[i]-arr[i])))
    yarr1=[]
    for i in range(0,len(X1)):
        for j in range(0,len(Y1)):
            if abs(X1[i]-Y1[j])==delayed[i]:
                yarr1.append(Y1[j])
            
    yarr2 = []
    for i in range(0,len(X1)):
        yarr2.append(list(Y.y_1[Y.time==yarr1[i]])[0])
    yarr2 = np.nan_to_num(yarr2)
    
    mse = mean_squared_error(yarr2,youtx)
    return mse

mse_11 = mse(df_ist_soll_1,yout_1,youto_11,to_11)
mse_21 = mse(df_ist_soll_2,yout_2,youto_21,to_21)
mse_31 = mse(df_ist_soll_3,yout_3,youto_31,to_31)
mse_41 = mse(df_ist_soll_4,yout_4,youto_41,to_41)
mse_12 = mse(df_ist_soll_1,yout_1,youto_12,to_12)
mse_22 = mse(df_ist_soll_2,yout_2,youto_22,to_22)
mse_32 = mse(df_ist_soll_3,yout_3,youto_32,to_32)
mse_42 = mse(df_ist_soll_4,yout_4,youto_42,to_42)
MSE1 = [mse_11,mse_21,mse_31,mse_41]
MSE2 = [mse_12,mse_22,mse_32,mse_42]

###### PT1 and PT2 Model response of all the dataset

In [ ]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
def f1(x):
    plt.plot(to1[x-1],youto1[x-1],label='pt1 model')
    plt.plot(t[x-1],yout[x-1],label='data')
    plt.legend()
    print('Mean square error is:')
    print(MSE1[x-1])
    return SYS1[x-1]
def f2(x):
    plt.plot(to2[x-1],youto2[x-1],label='pt2 model')
    plt.plot(t[x-1],yout[x-1],label='data')
    plt.legend()
    print('Mean square error is:')
    print(MSE2[x-1])
    return SYS2[x-1]
#interact(f1,x =widgets.IntSlider(min=1,max=4,step=1));
#interact(f2,x =widgets.IntSlider(min=1,max=4,step=1));
def sel(m):
    if m == 'pt1':
        #interact(f1,x =widgets.IntSlider(min=1,max=4,step=1));
        interact(f1,x = widgets.RadioButtons(options=[1, 2, 3, 4],value=1,description='Data:',disabled=False));
    else:
        #interact(f2,x =widgets.IntSlider(min=1,max=4,step=1));
        interact(f2,x = widgets.RadioButtons(options=[1, 2, 3, 4],value=1,description='Data:',disabled=False));
interact(sel,m = widgets.RadioButtons(options=['pt1', 'pt2'],value='pt1',description='System:',disabled=False));

###### Selecting the best model parameters among the dataset 

In [ ]:
for i in range(0,len(MSE1)):
    for j in range(0,len(MSE2)):
        if(MSE1[i] == min(MSE1)):
            i1 = i
        if(MSE2[j] == min(MSE2)):
            j1 = j
if MSE2[j1] < MSE1[i1]:
    plt.plot(to2[j1],youto2[j1],label='pt2 model')
    plt.plot(t[j1],yout[j1],label='data')
    plt.legend()
    print('The best model is a pt2 system and is obtained from dataset:',j1+1)
    print('Mean square error is:',MSE2[j1])
    print('The state space parameters are:')
    print(SYS2[j1])
else:
    plt.plot(to1[i1],youto1[i1],label='pt2 model')
    plt.plot(t[i1],yout[i1],label='data')
    plt.legend()
    print('The best model is a pt1 system and is obtained from dataset:',i1+1)
    print('Mean square error is:',MSE1[i1])
    print('The state space parameters are:')
    print(SYS1[i1])

In [ ]:
print(time.localtime())